In [1]:
# Import several packages that will be used throughout

# numeric packages
import numpy as np
import scipy
import scipy.io
import pandas as pd

# filesystem and OS
import sys, os, time
import glob

# images
import skimage
import skimage
import skimage.io

# plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

In [ ]:
def generate_map(L, T, Gu, Gr, Ue=0.2):
    
    Re = 1 - Ue # percentage of rural grow iteractions

    t = 0 # initialize percentage of filled surface

    # generate an URBAN seed
    Murb = np.zeros(L,L) # URBAN MATRIX
    seed = [round((L/2)),round((L/2))] # this will be updated every 10 iterations
    new_cities = sub2ind([L, L], seed(:,1), seed(:,2));

    % generate a rural urban pattern which is randomly distributed
    Mrur=Murb;
    [D,MW]=(bwdist(Murb));
    D(Murb<1) = (D(Murb<1).^exp(-D(Murb<1)./10));
    norm = sum(sum(D(Murb<1)))/500;
    D(Murb<1) = D(Murb<1)./norm;
    Mrur(rand(L,L)<D)=1;
    
    for it in range(maxiter):
        if t>=T # till a certain filled surface percentage
            break

        # URBAN GROW
        for i in range(Ue*10):
            Murb[new_cities] = 1 # generate a new attractor city
            [D,MW]=(bwdist(Murb));
            D(Murb<1)= D(Murb<1).^-Gu;
            norm = sum(sum(D(Murb<1)))/1;
            D(Murb<1) = D(Murb<1)./norm;
            Murb(rand(L,L)<D)=1;

        # RURAL GROW
        for e=1:(Re*10);
            %  if  RUR_r >0.1
            % second gamma RURAL
            [DI,MW]=(bwdist(Mrur));
            DI(Mrur<1)= DI(Mrur<1).^-Gr;
            norm = sum(sum(DI(Mrur<1)));
            DI(Mrur<1) = DI(Mrur<1)./norm;
            %LI=LO;
            Mrur(rand(L,L)<DI)=1;

            DII=Murb+Mrur;
        end
        % FINDING THE CENTROID OF NEW CITIES! this takes time so we can do
        % every 10 iterations
        if mod(e,10) == 0;
            CC = bwconncomp(DII);
            AA= regionprops(CC,'Area');
            f =find (vertcat(AA.Area)'>(TRE));
            new_cities=[];
            for n=1:length(f);
                new_cities = vertcat(new_cities,(cell2mat(CC.PixelIdxList(f(n)))));
            end


        end
         DII=Murb+Mrur;

        % percentage of filled surface to stop the iteration
        t = sum(sum(DII))/L^2;
        %t


        %PLOT


    end



end



In [ ]:
# MODEL OF SETTLEMENTS GROWTH

# Define the length of the  matrix given the resolution of the case
L = 300 # size of the inizial matrix
nnodes = L*L  # number of cells

# the iterations wich regulate G.

# PARAMETER FOR THE SIMULATION
maxiter=100000 

# PARAMETERS OF THE MODEL
Gu = 3   # Urban gamma
Gr = 2.2 # Rural gamma
TRE= 1   # threshold of area from rural to urban NOW IN NUMBER OF PIXEL

REL = np.linspace(20, 95, 10)
